In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def plot(grid):
    size = grid.shape[0]
    for i in range(size):
        for j in range(size):
            v = grid[i, j]
            if v <= 0.01:
                print(".", end=" ")
            elif v >= 0.99:
                print("X", end=" ")
            else:
                print(" ", end=" ")
                
        print("")

In [3]:
def append(l, r):
    if isinstance(r[0], list):
        return [append(l, i) for i in r]
    return [l] + r

def split(k, n):
    if k == 0:
        return [[0 for _ in range(n)]]
    if n == 1:
        return [[k]]
    L = []
    for left in range(k+1):
        right = split(k-left, n-1)
        for i in right:
            L.append(append(left, i))
    return L

def merge(a, b):
    # Assert a is the longest
    if len(b) > len(a):
        a, b = b, a
    L = []
    for i in range(len(b)):
        L.append(a[i])
        L.append(b[i])
    L.append(a[-1])
    return L

In [4]:
def toarray(s):
    return np.array([int(k) for k in s])

In [5]:
def possibilities(size, cons):
    l = ['1'*i for i in cons]
    s = "0".join(l)
    n_missing = size - len(s)
    if n_missing == 0:
        return [s]
    n_inter = len(l) + 1
    filling = split(n_missing, n_inter)
    L = []
    for i in range(len(l)-1):
        l[i]=l[i]+"0"
    for k in filling:
        k = ["0"*i for i in k]
        sol = "".join(merge(l, k))
        L.append(sol)
    return L

def get_pos(size, cons):
    return [toarray(sol) for sol in possibilities(size, cons)]

In [6]:
for i in possibilities(10, [3, 4]):
    print(i)
    print(len(i))

1110111100
10
1110011110
10
1110001111
10
0111011110
10
0111001111
10
0011101111
10


In [7]:
def apply_filter(L, val):
    val = np.array(val)
    undecided = (val != 0) * (val != 1)
    filtered_L = []
    for l in L:
        f = (val == l) + undecided
        if np.all(f):
            filtered_L.append(l)
    return filtered_L

In [8]:
def get_probas(l):
    L = np.array([toarray(i) for i in l])
    p = np.mean(L, axis=0)
    return p

In [9]:
def step(grid, rows, columns):
    size = len(rows)
    # Rows
    for i in range(size):
        val = grid[i, :]
        c = rows[i]
        pos = get_pos(size, c)
        avail_pos = apply_filter(pos, val)
        grid[i, :] = get_probas(avail_pos)
    
    # Columns
    for i in range(size):
        val = grid[:, i]
        c = columns[i]
        pos = get_pos(size, c)
        avail_pos = apply_filter(pos, val)
        grid[:, i] = get_probas(avail_pos)
    
    return grid

In [10]:
def solve(cfg, max_steps=10):
    size = cfg["size"]
    grid = np.ones((size, size))*0.5

    n_steps = 0 
    while not np.all((grid == 0) + (grid == 1)):
        n_steps += 1
        if n_steps > max_steps:
            print("Max steps reached")
            return grid
        grid = step(grid, cfg["rows"], cfg["columns"])

    print(f"Found in {n_steps} steps")
    return grid

In [11]:
def read_config(path):
    file = open(path, 'r')
    Lines = file.readlines()
    size = int(Lines[0].replace("\n", "").replace("Size ", ""))
    config = {
        "size":size,
        "rows":[],
        "columns":[]
    }
    cat = ""
    for l in Lines:
        if "rows" in l.lower():
            cat = "rows"
        elif "columns" in l.lower():
            cat = "columns"
        else:
            if "," in l:
                v = [int(i) for i in l.split(",")]
                assert sum(v) + len(v)-1 <= size, "Impossible constraints"
                config[cat].append(v)
            else:
                try:
                    v = int(l)
                    assert v <= size, "Impossible constraints"
                    config[cat].append([v])
                except:
                    continue
    assert len(config["rows"]) == config["size"], "Not enough rows"
    assert len(config["columns"]) == config["size"], "Not enough columns"
    return config

In [12]:
cfg = read_config("grid_10.txt")
size = cfg["size"]
grid = np.ones((size, size))*0.5
step(grid, cfg["rows"], cfg["columns"])

array([[0.125     , 0.        , 1.        , 1.        , 1.        ,
        0.5       , 0.5       , 0.        , 0.5       , 0.        ],
       [0.125     , 0.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 0.44444444],
       [0.5       , 0.5       , 1.        , 1.        , 1.        ,
        0.5       , 1.        , 1.        , 0.5       , 0.77777778],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        0.        , 1.        , 1.        , 0.        , 1.        ],
       [0.5       , 1.        , 0.        , 0.        , 0.        ,
        0.5       , 1.        , 1.        , 1.        , 0.55555556],
       [0.125     , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 0.33333333],
       [0.375     , 1.        , 0.        , 0.        , 0.        ,
        0.5       , 0.5       , 1.        , 1.        , 0.33333333],
       [0.5       , 1.        , 0.       

In [13]:
cfg = read_config("grid_10.txt")
print(cfg)
grid = solve(cfg)
plot(grid)

{'size': 10, 'rows': [[4], [7], [3, 4], [5, 2, 1], [2, 4], [8], [1, 4], [2, 3], [3, 5], [8]], 'columns': [[2, 2], [7], [4, 1, 2], [4, 1, 1], [4, 1, 1], [2, 2, 2], [6, 2], [9], [2, 6], [3, 2]]}
Found in 3 steps
. . X X X X . . . . 
. . X X X X X X X . 
. . X X X . X X X X 
X X X X X . X X . X 
X X . . . . X X X X 
. X X X X X X X X . 
. X . . . X X X X . 
X X . . . . . X X X 
X X X . . X X X X X 
. X X X X X X X X . 


In [20]:
cfg = read_config("grid_15.txt")
print(cfg)
grid = solve(cfg)
plot(grid)

{'size': 15, 'rows': [[11], [4, 3, 3], [1, 7, 1], [11], [3, 4], [14], [2, 6], [2, 4, 5], [2, 5, 5], [4, 2, 3], [3, 2, 5], [4, 2, 3, 1], [13], [3, 4], [9]], 'columns': [[2, 6], [1, 9], [2, 3, 5], [2, 3, 2, 4], [1, 2, 1, 2, 3], [1, 2, 1, 2, 3, 1], [4, 1, 2, 3, 1], [4, 1, 3, 1, 1], [4, 1, 2, 1, 1], [1, 2, 2, 5], [1, 7, 5], [1, 12], [2, 8, 2], [1, 6, 2], [2, 4, 1]]}
Found in 3 steps
. . X X X X X X X X X X X . . 
X X X X . . X X X . . . X X X 
X . . . X X X X X X X . . . X 
. . X X X X X X X X X X X . . 
. X X X . . . . . . X X X X . 
. X X X X X X X X X X X X X X 
X X . . . . . . . X X X X X X 
X X . . X X X X . . X X X X X 
X X . X X X X X . . X X X X X 
X X X X . . . X X . . X X X . 
X X X . . X X . X X X X X . . 
X X X X . X X . . X X X . X X 
. X X X X X X X X X X X X X . 
. . X X X . . . . X X X X . . 
. . . X X X X X X X X X . . . 


In [21]:
v = []

In [22]:
v[0]="0000001001110"
v[1]="0000011101010"
v[2]="0000111111010"
v[3]="0001111111110"
v[4]="0111111111110"
v[5]="1111111111111"
v[6]="0100000000010"
v[7]="0101110000010"
v[8]="0101010011110"
v[9]="0101110010010"
v[10]="0100000010110"
v[11]="0100000010010"
v[12]="0100000010010"
v[13]="0111111111110"

In [23]:
v

array([1.00111000e+06, 1.11010100e+07, 1.11111010e+08, 1.11111111e+09,
       1.11111111e+11, 1.11111111e+12, 1.00000000e+11, 1.01110000e+11,
       1.01010011e+11, 1.01110010e+11, 1.00000010e+11, 1.00000010e+11,
       1.00000010e+11, 1.11111111e+11])

In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [16]:
url = "https://fr.goobix.com/jeux-en-ligne/nonograms/?s=25"

In [17]:
html = requests.get(url).text

In [18]:
s = html.split("nonogramsDef")[1]

In [19]:
l = s.split("<strong>")